# Layer: Base

yardw: is there a way to lazily init a matrix by a function f? f is very costly so I only want to call it when it is really needed

GitHub Copilot: Yes, in Julia, you can create a lazy matrix using the `getindex` function. Here's an example of how you can do it:

```julia
struct LazyMatrix
    f::Function
    rows::Int
    cols::Int
end

Base.size(A::LazyMatrix) = (A.rows, A.cols)

function Base.getindex(A::LazyMatrix, i::Int, j::Int)
    A.f(i, j)
end
```

In this code, `LazyMatrix` is a type that stores a function `f` and the dimensions of the matrix. When you try to access an element of the matrix, the `getindex` function is called, which in turn calls the function `f`. This way, the function `f` is only called when an element of the matrix is accessed.

Here's how you can use it:

```julia
f(i, j) = i + j  # replace this with your costly function
A = LazyMatrix(f, 1000, 1000)
println(A[1, 1])  # f is called here
```

In this example, `f` is only called when you access an element of `A`.

yardw: I want to memorize the element if it is accessed

GitHub Copilot: To memorize the elements of the matrix once they are accessed, you can modify the `LazyMatrix` structure to include a cache. Here's how you can do it:

```julia
struct LazyMatrix
    f::Function
    rows::Int
    cols::Int
    cache::Dict{Tuple{Int, Int}, Any}
end

LazyMatrix(f::Function, rows::Int, cols::Int) = LazyMatrix(f, rows, cols, Dict{Tuple{Int, Int}, Any}())

Base.size(A::LazyMatrix) = (A.rows, A.cols)

function Base.getindex(A::LazyMatrix, i::Int, j::Int)
    if (i, j) in A.cache
        return A.cache[(i, j)]
    else
        result = A.f(i, j)
        A.cache[(i, j)] = result
        return result
    end
end
```

In this code, `LazyMatrix` now includes a `cache` field, which is a dictionary that stores the results of the function `f` for each pair of indices. When you try to access an element of the matrix, the `getindex` function first checks if the result is in the cache. If it is, it returns the cached result. If it's not, it calls the function `f`, stores the result in the cache, and then returns the result.

Here's how you can use it:

```julia
f(i, j) = i + j  # replace this with your costly function
A = LazyMatrix(f, 1000, 1000)
println(A[1, 1])  # f is called here
println(A[1, 1])  # f is not called here, the result is retrieved from the cache
```

In this example, `f` is only called the first time you access an element of `A`. If you access the same element again, the result is retrieved from the cache.

yardw: is the performance of dict as good as a real matrix with  a random access?

GitHub Copilot: In Julia, accessing elements in a `Dict` and a matrix (2D array) have different performance characteristics.

Accessing an element in a `Dict` is generally an O(1) operation, meaning it takes constant time regardless of the size of the `Dict`. However, this is an average case scenario. In the worst case (when all keys hash to the same index), lookup can degrade to O(n), where n is the number of elements in the `Dict`.

On the other hand, accessing an element in an array is always an O(1) operation, as it directly computes the memory address of the element.

So in general, if you're doing a lot of random accesses, a matrix (2D array) will be faster than a `Dict`. However, the `Dict` has the advantage of being sparse: it only needs to store the elements that have been accessed, whereas the matrix needs to allocate memory for all elements upfront.

In the case of your `LazyMatrix`, if the function `f` is very costly and most elements of the matrix are never accessed, then using a `Dict` could be more efficient overall, despite the slower access time.

yardw: does the allocation of the dict slow down the program a lot?

GitHub Copilot: The allocation of a `Dict` in Julia, or any other dynamic data structure, does have a cost associated with it. However, whether this significantly slows down your program depends on the specifics of your use case.

If the `Dict` is large and you're frequently creating, resizing, or deleting it, then these operations could potentially become a bottleneck. However, if the `Dict` is relatively small or its size remains constant over the program's execution, then the cost of these operations is likely to be negligible compared to other parts of your program, especially if those parts involve complex computations.

In the case of your `LazyMatrix`, the `Dict` is used as a cache to avoid recomputing the costly function `f`. The cost of allocating and accessing the `Dict` could be outweighed by the savings from not having to recompute `f`.

If you're concerned about performance, the best approach is to profile your code to identify where the most time is being spent. Julia provides built-in tools for profiling, such as the `@profile` macro. This will give you a better idea of whether the `Dict` operations are a significant factor in your program's execution time.


In [ ]:
struct LazyMatrix{T} <: AbstractMatrix{T} where T
    f::Function
    rows::Int
    cols::Int
    cache::Dict{Tuple{Int, Int}, T}
end

LazyMatrix(f::Function, rows::Int, cols::Int) = LazyMatrix(f, rows, cols, Dict{Tuple{Int, Int}, Any}())

Base.size(A::LazyMatrix) = (A.rows, A.cols)

function Base.getindex(A::LazyMatrix, i::Int, j::Int)
    if (i, j) in A.cache
        return A.cache[(i, j)]
    else
        result = A.f(i, j)
        A.cache[(i, j)] = result
        return result
    end
end

function Base.axes(A::LazyMatrix)
    x_axis = 1:A.rows
    y_axis = 1:A.cols
    return x_axis, y_axis
end

In [ ]:
f(i, j) = i + j  # replace this with your costly function
A = LazyMatrix(f, 1000, 1000)
x_axis, y_axis = axes(A)
println(A[x_axis[1], y_axis[1]])  # f is called here

In [ ]:
"""
    responsibilitychain(handlers::Vector{Function})::Function 

Generate a Chain of Responsibility, which return a function that deal with a keyword request by handlers in order.
Any handler is a of function, which take a request and return a response or nothing.
If a handler return a response, the response will be returned and the rest handlers will be skipped.
If a handler return nothing, the next handler will be called.
# Examples
```jldoctest
julia> function handler1(; request...)
           println("handler1")
           return nothing
       end
julia> function handler2(; request...)
           println("handler2")
           return "response"
       end
julia> function handler3(; request...)
           println("handler3")
           return "unexpected response"
       end
julia> handle = responsibilitychain([handler1, handler2, handler3])
julia> handle()
"handler1"
"handler2"
"response"
```
# Tricky Realization
```jldoctest
julia> handle = foldl(||, handlers)
```
- in this case, simple handlers return either true or false. If a handler return true, the remaining handlers will be skipped because true || anything is true.
- in this case, no response can be returned.
"""
function responsibilitychain(handlers::Vector{Function})::Function
    function handle(; request...)
        for handler in handlers
            response = handler(; request...)
            isnothing(response) ? continue : return response
        end
    end
    return handle
end


In [ ]:
abstract type Graph end
abstract type GraphStyle end
struct UnknownGraphStyle <: GraphStyle end
function adjacent end
function priority end
style(g::Graph) = UnknownGraphStyle()
function adjacent(::UnknownGraphStyle)
    @error "UnknownGraphStyle"
end
Base.BroadcastStyle

> No virtual methods support, since for scientific computing, the performance penalty of virtual methods is too high.

> However, dispatch + type comprehension = virtual methods. One example is the broadcast functions in Base.
- Details:
    - 

In [ ]:
"""
The threading macro is like a more flexible version of the [`|>`](@ref) operator.

    @> x f = f(x)
    @> x g f == f(g(x))
    @> x a b c d e == e(d(c(b(a(x)))))

Unlike [`|>`](@ref), functions can have arguments - the value
preceding a function will be treated as its first argument

    @> x g(y, z) f == f(g(x, y, z))

    @> x g f(y, z) == f(g(x), y, z)

See Lazy.jl.
"""
macro >(exs...)
    thread(x) = isexpr(x, :block) ? thread(rmlines(x).args...) : x
  
    @static if VERSION < v"0.7"
  
      thread(x, ex) =
      isexpr(ex, :call, :macrocall) ? Expr(ex.head, ex.args[1], x, ex.args[2:end]...) :
      @capture(ex, f_.(xs__))       ? :($f.($x, $(xs...))) :
      isexpr(ex, :block)            ? thread(x, rmlines(ex).args...) :
      Expr(:call, ex, x)
  
    else
  
      thread(x, ex) =
      isexpr(ex, :macrocall)        ? Expr(ex.head, ex.args[1], ex.args[2], x, ex.args[3:end]...) :
      isexpr(ex, :call,)            ? Expr(ex.head, ex.args[1], x, ex.args[2:end]...) :
      @capture(ex, f_.(xs__))       ? :($f.($x, $(xs...))) :
      isexpr(ex, :block)            ? thread(x, rmlines(ex).args...) :
      Expr(:call, ex, x)
  
    end
  
    thread(x, exs...) = reduce(thread, exs, init=x)
  
    esc(thread(exs...))
  end
  

# Layer: Logic

# Layer: Application

# Layer: Test

## dependency test

In [56]:
using Test
@testset "find all methods mathing arguments" begin
    # methods returns a vector of methods matching arguments
    @test first(methods(+, (Int, Int))) == @which 1 + 1 
    # if no method matches, methods returns an empty vector
    @test isempty(methods(+, (Int, String)))
    # typeof can be used to get the type of given arguments
    @test isempty(methods(
        +, typeof.((1,"a"))
        ))
    # test a user defined function which takes keyword arguments
    foo(dumb; a::Int=nothing, b) = a + b
    method = first(methods(foo))
    @show Base.method_argnames(method)
    # @test method == @which foo(;a=1, b=2)
    @show Base.kwarg_decl(method)
    # @show keys((;a=1, b=2))
    # @show eltype((;a=1, b=2))
    # @show methods(foo)
    # args = (; a=1, b=2)
    # @show methods(foo, (;typeof(args)...))
    # @show @which foo(;a=1, b=2)
    # @test first(methods(foo, @show typeof((;a=1, b=2))) ) == @show @which foo(;a=1, b=2)
end

Base.method_argnames(method) = [Symbol("#self#"), :dumb]
Base.kwarg_decl(method) = [:a, :b]
Test Summary:                      | Pass  Total  Time
find all methods mathing arguments |    3      3  0.0s


Test.DefaultTestSet("find all methods mathing arguments", Any[], 3, false, false, true, 1.704099050096e9, 1.704099050108e9, false)

In [57]:
@testset "can for loop return a value?" begin
    function foo()
        n = for i in 1:10
            if i == 5
                break
            end
        end
    end
    @test foo() == 5
end

can for loop return a value?: Test Failed at l:\Sieg\Doc\Code\julia\HierarchicalSearches.jl\dev.ipynb:9
  Expression: foo() == 5
   Evaluated: nothing == 5

Stacktrace:
 [1] macro expansion
   @ C:\Users\Acer\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\Test\src\Test.jl:478 [inlined]
 [2] macro expansion
   @ l:\Sieg\Doc\Code\julia\HierarchicalSearches.jl\dev.ipynb:9 [inlined]
 [3] macro expansion
   @ C:\Users\Acer\AppData\Local\Programs\Julia-1.9.0\share\julia\stdlib\v1.9\Test\src\Test.jl:1498 [inlined]
 [4]

 top-level scope
   @ l:\Sieg\Doc\Code\julia\HierarchicalSearches.jl\dev.ipynb:2
Test Summary:                | Fail  Total  Time
can for loop return a value? |    1      1  0.2s


TestSetException: Some tests did not pass: 0 passed, 1 failed, 0 errored, 0 broken.

In [59]:
using Lazy
@> 1 +(1)

2

## development test

In [ ]:
abstract type Graphs end

In [ ]:
struct TopoGraphs <: Graphs
    vertices::Set{Int}
    edges::Dict{Int, Set{Int}}
end
import Base: getindex
getindex(g::TopoGraphs, i::Int) = g.edges[i]
function dfs(graph::TopoGraphs, start::Int)
    visited = Set{Int}()
    sizehint!(visited, length(graph.vertices))
    stack = [start]
    while !isempty(stack)
        node = pop!(stack)
        if node in visited continue end
        push!(visited, node)
        for neighbor in graph[node]
            push!(stack, neighbor)
        end
    end
    return visited
end
LazyArrays

In [7]:
using Test
@testset "dfs tests" begin
    g = TopoGraphs(Set([1, 2, 3, 4, 5]), Dict(1 => Set([2, 3]), 2 => Set([4]), 3 => Set([5]), 4 => Set([]), 5 => Set([])))

    @testset "dfs from vertex 1" begin
        # Assuming dfs returns a list of visited vertices in the order they were visited
        @test dfs(g, 1) == Set([5, 4, 2, 3, 1])
    end

    @testset "dfs from vertex 2" begin
        @test dfs(g, 2) == Set([2, 4])
    end

    @testset "dfs from vertex 3" begin
        @test dfs(g, 3) == Set([3, 5])
    end
end

Test Summary: | Pass  Total  Time


dfs tests     |    3      3  0.0s


Test.DefaultTestSet("dfs tests", Any[Test.DefaultTestSet("dfs from vertex 1", Any[], 1, false, false, true, 1.704047495808e9, 1.704047495833e9, false), Test.DefaultTestSet("dfs from vertex 2", Any[], 1, false, false, true, 1.704047495833e9, 1.704047495833e9, false), Test.DefaultTestSet("dfs from vertex 3", Any[], 1, false, false, true, 1.704047495833e9, 1.704047495833e9, false)], 0, false, false, true, 1.704047495808e9, 1.704047495833e9, false)

In [5]:
struct LatticeGraphs <: Graphs
    lattice::AbstractMatrix
end
import Base: getindex
getindex(g::LatticeGraphs, i::CartesianIndex) = g.lattice[i]

Set{Int64} with 3 elements:
  2
  3
  1